In [0]:
# cotacao do preço do Etanol Hidratado Comum (EHC)

# preço do açucar

# producao de safra de cana de acucar

# cambio real - dolar

# preço do petróleo/gasolina

# media de tempeatura nas regioes produtoras

# media de volume de chuva nas regioes produtoras

# volume de exportação

